<a href="https://colab.research.google.com/github/mingmingbupt/tensorflow/blob/master/tf_keras_regression_manu_diffs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.2.0-rc1
sys.version_info(major=3, minor=6, micro=9, releaselevel='final', serial=0)
matplotlib 3.2.1
numpy 1.18.2
pandas 0.25.3
sklearn 0.22.2.post1
tensorflow 2.2.0-rc1
tensorflow.python.keras.api._v2.keras 2.2.4-tf


In [2]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
print(housing.DESCR)
print(housing.data.shape)
print(housing.target.shape)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block
        - HouseAge      median house age in block
        - AveRooms      average number of rooms
        - AveBedrms     average number of bedrooms
        - Population    block population
        - AveOccup      average house occupancy
        - Latitude      house block latitude
        - Longitude     house block longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
http://lib.stat.cmu.edu/datasets/

The target variable is the median house value for California districts.

This dataset was derived from the 1990 U.S. census, using one row per census
block group. A block group is the smallest geographical unit for which the U.S.
Census Bur

In [3]:
from sklearn.model_selection import train_test_split

x_train_all, x_test, y_train_all, y_test = train_test_split(
    housing.data, housing.target, random_state = 7)
x_train, x_valid, y_train, y_valid = train_test_split(
    x_train_all, y_train_all, random_state = 11)
print(x_train.shape, y_train.shape)
print(x_valid.shape, y_valid.shape)
print(x_test.shape, y_test.shape)


(11610, 8) (11610,)
(3870, 8) (3870,)
(5160, 8) (5160,)


In [0]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_valid_scaled = scaler.transform(x_valid)
x_test_scaled = scaler.transform(x_test)

In [8]:
# metric使用
# 这里使用的metric是均方差，即差的平方和的均值
# 看下使用例子

metric = keras.metrics.MeanSquaredError()
print(metric([5.], [2.])) # 直接像函数一样去对他进行使用
print(metric([0.], [1.])) # 输入是两个列表。真实列表和预测列表
print(metric.result()) #keras里面的metric具有累加功能的，可以通过调用多次看下
#这是metric的一个特性，就是它可以累加数据
#如果不想累加呢，我们只需要调用metric.reset_states()就可以了
#这样他就会把之前记录结果全部清空

metric.reset_states()
metric([1.], [3.])
print(metric.result()) # 之前结果清空了
#我们可以在每个epoch结束的时候调用下reset_states，然后再去统计在这里epoch内，训练集上面的结果

tf.Tensor(9.0, shape=(), dtype=float32)
tf.Tensor(5.0, shape=(), dtype=float32)
tf.Tensor(5.0, shape=(), dtype=float32)
tf.Tensor(4.0, shape=(), dtype=float32)


In [9]:
# 需要改的就是model.fit部分，fit函数部分都做了什么呢？
# 1. 按照batch的形式呢，去遍历训练集 ，在每一次初始化去训练的时候呢，都会得到一个指标，也就是loss. 然后是要统计metric
#    1.1 在训练的时候呢，会自动求导，去更新参数。在这里我们想要替换的是这一部分，因为他是由一个fit函数去封装的，所以呢
#      我们需要替换这一部分的话，这三部分都要自己实现
# 2. 然后在一次所有batch训练完以后呢，相当于一次epoch结束，这时我们会去验证集进行验证，统计出在验证集的metric


epochs = 100   #执行100次
batch_size = 32
steps_per_epoch = len(x_train_scaled) // batch_size #在每个epoch训练多少次
optimizer = keras.optimizers.SGD()  #使用sgd,使用默认的learning_rate
metric = keras.metrics.MeanSquaredError() #使用mse

#如何去取数据呢，在fit函数里面呢，他应该是随机的去遍历数据，也就是我把数据随机的shuffle一次，然后呢从头到尾去遍历
#这里呢，我们就使用一种更简洁的实现方法，去进行数据的遍历，他并不是严格的数据遍历，因为我们这里实现的方式呢，是
#随机的去取，即我每次就从训练集里随机的取batch_size条样本就可以了
def random_batch(x, y, batch_size=32):
    idx = np.random.randint(0, len(x), size=batch_size) #先随机出来batch_size个索引，用numpy里面的random函数取出从最低值到最高值的32个数
    #0是最小值，len(x)就是他的最大值，这是一个半闭半开区间，所以取出来的值不会超过len(x)
    return x[idx], y[idx] #然后呢，在x和y上把索引对应的数据给返回出来就可以了，因为x和y都是numpy类型，所以可以这样取数据。idx其实是个列表

#模型没变
model = keras.models.Sequential([
    keras.layers.Dense(30, activation='relu',
                       input_shape=x_train.shape[1:]),
    keras.layers.Dense(1),
])

#实现上面说的3步
for epoch in range(epochs):  #遍历epochs
    metric.reset_states()  #在每个epoch的开端呢，都要把metrics给reset下
    for step in range(steps_per_epoch): # 在每个epoch里面，我都要循环steps_per_epoch次
        x_batch, y_batch = random_batch(x_train_scaled, y_train, batch_size)
        # 在每次进行训练的时候呢，我都要取出数据来，取出来batch_size个
        # 然后再取出这些数据以后呢，我们就可以获取这些数据的预测值了

        with tf.GradientTape() as tape: #求梯度需要打开一个tf.GradientTape()
            y_pred = model(x_batch) #在这里呢，可以把model像函数一样使用，他的输入呢是x_batch，输出就是他的预测值
            y_pred = tf.squeeze(y_pred, 1)
            loss = keras.losses.mean_squared_error(y_batch, y_pred) #有了预测值以后就可以得到他的loss,这里使用mse
            metric(y_batch, y_pred) #有了真实值和预测值，可以累计的去计算metric
            #这里相当于我们的目标函数loss就setup好了，接下来我们去求梯度

        grads = tape.gradient(loss, model.variables) #手动求梯度，参数是model的所有参数
        grads_and_vars = zip(grads, model.variables) #把梯度和变量呢，一一绑定
        optimizer.apply_gradients(grads_and_vars) #绑定以后，就可以像上个文件一样，optimizer.apply_gradients，去更新参数啦
        print("\rEpoch", epoch, " train mse:",
              metric.result().numpy(), end="") #然后打印下，把训练的mse打印出来，他是一个训练的累计的值
    y_valid_pred = model(x_valid_scaled) #在每个epoch训练完以后，就可以在验证机上进行验证啦，
    y_valid_pred = tf.squeeze(y_valid_pred, 1) 
    valid_loss = keras.losses.mean_squared_error(y_valid_pred, y_valid)
    print("\t", "valid mse: ", valid_loss.numpy()) #在这里只用了一个loss
        


ERROR! Session/line number was not unique in database. History logging moved to new session 59
Epoch 0  train mse: 0.76856667	 valid mse:  0.6498970384363162
Epoch 1  train mse: 0.7825577	 valid mse:  0.6805622780792883
Epoch 2  train mse: 0.48554593	 valid mse:  0.48823736910830723
Epoch 3  train mse: 0.45413145	 valid mse:  0.47299750216828235
Epoch 4  train mse: 0.41907844	 valid mse:  0.4516531804336801
Epoch 5  train mse: 0.43206713	 valid mse:  0.4353824332214284
Epoch 6  train mse: 0.40812662	 valid mse:  0.4278446420896621
Epoch 7  train mse: 0.39901805	 valid mse:  0.4231546313701608
Epoch 8  train mse: 0.38326445	 valid mse:  0.4132924501046734
Epoch 9  train mse: 0.66875666	 valid mse:  0.5106283670573705
Epoch 10  train mse: 0.45630127	 valid mse:  0.4603723174451882
Epoch 11  train mse: 0.43282712	 valid mse:  0.4379744532578509
Epoch 12  train mse: 0.42354086	 valid mse:  0.4221945263659818
Epoch 13  train mse: 0.4060955	 valid mse:  0.4179349549677343
Epoch 14  train mse